In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt, seaborn as sb

In [ ]:
df = pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:

df["bmi"].fillna(28.893237,inplace=True)


In [ ]:
df["ever_married"].value_counts()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
sb.boxplot(x=df["bmi"])


In [ ]:
sb.boxplot(df["avg_glucose_level"])


In [ ]:
df["avg_glucose_level"].values

In [ ]:
q1 = df["bmi"].quantile(0.25)
q3 = df["bmi"].quantile(0.75)
q1,q3

In [ ]:
iqr = q3-q1
iqr

In [ ]:
lower_limit = q1 - 1.5*iqr
upper_limit = q3 + 1.5*iqr
upper_limit

In [ ]:
df = df[(df["bmi"] >= lower_limit) & (df["bmi"] <= upper_limit)]


In [ ]:
df.shape

In [ ]:
q1 = df["avg_glucose_level"].quantile(0.25)
q3 = df["avg_glucose_level"].quantile(0.75)
q1,q3

In [ ]:
iqr=q3-q1
iqr

In [ ]:
lower_limit = q1 - 1.5*iqr
upper_limit = q3 + 1.5*iqr
upper_limit

In [ ]:
df = df[(df["avg_glucose_level"] >= lower_limit) & (df["avg_glucose_level"] <= upper_limit)]

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()


In [ ]:
dummies = pd.get_dummies(df["gender"])
dummies

In [ ]:
df=pd.concat([df,dummies],axis=1)
df

In [ ]:
df.drop(["gender","Other","id"],axis=1,inplace=True)
df

In [ ]:
dummies = pd.get_dummies(df["ever_married"])
dummies

In [ ]:
df=pd.concat([df,dummies],axis=1)
df

In [ ]:
df.drop(["ever_married","No"],axis=1,inplace=True)
df

In [ ]:
dummies = pd.get_dummies(df["work_type"])
dummies

In [ ]:
df=pd.concat([df,dummies],axis=1)
df

In [ ]:
df.drop(["work_type","children"],axis=1,inplace=True)
df

In [ ]:
dummies = pd.get_dummies(df["Residence_type"])
dummies

In [ ]:
df=pd.concat([df,dummies],axis=1)
df

In [ ]:
df.drop(["Residence_type","Rural"],axis=1,inplace=True)
df

In [ ]:
dummies = pd.get_dummies(df["smoking_status"])
dummies

In [ ]:
df=pd.concat([df,dummies],axis=1)
df

In [ ]:
df.drop(["smoking_status","Unknown"],axis=1,inplace=True)
df

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [ ]:
num_vars = ["avg_glucose_level","bmi","age"]

df[num_vars] = scaler.fit_transform(df[num_vars])

In [ ]:
df

In [ ]:
x = df.drop("heart_disease",axis=1)
y = df["heart_disease"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state = 100)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
model_param = {"DesicionTreeClassifier":{"model": DecisionTreeClassifier(),"param":{"criterion":["gini","entropy"]}},
               "KNeighborsClassifier":{"model":KNeighborsClassifier(),"param":{"n_neighbors":[5,10,15,20,25]}},
               "SVC":{"model":SVC(),"param":{"kernel":["rbf","linear","sigmoid"],"C":[0.1,1,10,100]}},
               "LogisticRegression":{"model":LogisticRegression(),"param":{"random_state" : [0,10,15],"solver" : ["liblinear","lbfgs"],"multi_class":["ovr","multinomial","auto"],"C":[1,5,10]}}
}

In [ ]:
scores=[]
for model_name,  mp in model_param.items():
        model_selection =  GridSearchCV(mp['model'],mp['param'], cv=5, return_train_score=False)
        model_selection.fit(x_train,y_train)
        scores.append({
            'model': model_name,
            'best_score': model_selection.best_score_,
            'best_params': model_selection.best_params_
        })

In [ ]:
df2 = pd.DataFrame(scores,columns=["model","best_score","best_params"])
df2.head()

In [ ]:
model_clf=LogisticRegression(C=1,multi_class="ovr",random_state=10)
model_clf.fit(x_train,y_train)

In [ ]:
model_clf.score(x_train,y_train)

In [ ]:
y_predicted=model_clf.predict(x_test)

In [ ]:
model_clf.score(x_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_predicted)
cm

In [ ]:
sb.heatmap(cm,annot=True)

In [ ]:
model_svc = SVC(C=0.1,kernel = "rbf")
model_svc.fit(x_train,y_train)

In [ ]:
y_predicted=model_svc.predict(x_test)
y_predicted

In [ ]:
model_svc.score(x_train,y_train)

In [ ]:
model_svc.score(x_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_predicted)
cm

In [ ]:
sb.heatmap(cm,annot=True)

In [ ]:
model_tree = DecisionTreeClassifier(criterion="entropy")
model_tree.fit(x_train,y_train)

In [ ]:
y_predicted=model_tree.predict(x_test)
y_predicted

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predicted)

In [ ]:
model_tree.score(x_test,y_test)

In [ ]:
model_tree.score(x_train,y_train)

In [ ]:
from sklearn import tree
plt.figure(figsize=(50,50))
tree.plot_tree(model_tree.fit(x_train,y_train))
plt.show()

In [ ]:
cm=confusion_matrix(y_test,y_predicted)
cm

In [ ]:
sb.heatmap(cm,annot=True)

In [ ]:
from sklearn.metrics import classification_report

classification_report(y_test,y_predicted)
